In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os


In [ ]:
#works perfectly for downsizing image can be used for lower res images also but will not bring dimensions to 300x300
def downsizingimg(folderpath: str,imgname: str):
  img = Image.open(folderpath+imgname)
  img.thumbnail((300,300))
  img.save(folderpath[:-1]+'_/'+imgname)
  m,n = img.size
  target_size = (300, 300)
  if (m!=300)or(n!=300):
    if m>n:
      target_size = (m,m)
    else:
      target_size = (n,n)
  else:
    return;
  image = cv2.imread(folderpath[:-1]+'_/'+imgname)
  vertical_padding = (target_size[0] - image.shape[0]) // 2
  horizontal_padding = (target_size[1] - image.shape[1]) // 2

  padded_image = cv2.copyMakeBorder(image, vertical_padding, vertical_padding, horizontal_padding, horizontal_padding, cv2.BORDER_CONSTANT)
  cv2.imwrite(folderpath[:-1]+'_/'+imgname,padded_image)


  #brings image dims exactly to 300x300
#run after downsizingimg()
def resize_image(image_path):
    img = load_img(image_path, target_size=(300, 300), color_mode='grayscale')
    img_array = img_to_array(img)
    img_array /= 255.0  # Normalize pixel values to between 0 and 1
    return img_array

In [ ]:
def create_model():
  model = models.Sequential()
  model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(300, 300, 1)))
  model.add(layers.MaxPooling2D((4, 4)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(32, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))

  model.add(layers.Flatten())

  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(16, activation='relu'))

  model.add(layers.Dense(2, activation='softmax'))


  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model
model = create_model()
model.summary()

In [ ]:
metadatapath = "updated.csv"
cancerpath = "Brain Tumor_/"
normalpath = "Healthy_/"

metadata = pd.read_csv(metadatapath)

paths=[]
labels=[]

metadata.drop(metadata.columns[[0]], axis=1, inplace=True)

for i in range(0,4519):
  if (metadata.iloc[i]['class']=='cancer' ):
    paths.append(cancerpath+metadata.iloc[i]['Filename'])
    labels.append('cancer')
  else:
    paths.append(normalpath+metadata.iloc[i]['Filename'])
    labels.append('normal')
df = pd.DataFrame({
    'path':paths,
    'label':labels
})

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='path',
    y_col='label',
    target_size=(300, 300),
    batch_size=4,
    class_mode='binary',  # or 'binary' for binary classification
    color_mode='grayscale',  # or 'rgb' for RGB images
    shuffle=True
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='path',
    y_col='label',
    target_size=(300, 300),
    batch_size=1,
    class_mode='binary',
    color_mode='grayscale',
    shuffle=False
)
val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='path',
    y_col='label',
    target_size=(300, 300),
    batch_size=1,
    class_mode='binary',
    color_mode='grayscale',
    shuffle=False
)


In [ ]:
#checkpoint = ModelCheckpoint('model10.h5', save_best_only=True)
#history = 
model.fit(
    train_generator,
    epochs=1,
    validation_data=val_generator
    #callbacks=[checkpoint]
)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.save("mod.h5")